In [12]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [13]:
address = 'Kansas City Metropolitan Area'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kansas City Metropolitan Area are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kansas City Metropolitan Area are 39.0694616, -94.5560807163324.


##### Category IDs corresponding to Gym / Fitness Center were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

### Gym / Fitness Center : 4bf58dd8d48988d175941735
#### SubCategories :
-  Boxing Gym : 52f2ab2ebcbc57f1066b8b47
-  Climbing Gym : 503289d391d4c4b30a586d6a
-  Cycle Studio : 52f2ab2ebcbc57f1066b8b49
-  Gym Pool : 4bf58dd8d48988d105941735
-  Gymnastics Gym : 52f2ab2ebcbc57f1066b8b48
-  Gym : 4bf58dd8d48988d176941735
-  Martial Arts Dojo : 4bf58dd8d48988d101941735
-  Outdoor Gym : 58daa1558bbb0b01f18ec203
-  Pilates Studio : 5744ccdfe4b0c0459246b4b2
-  Track : 4bf58dd8d48988d106941735
-  Weight Loss Center : 590a0744340a5803fd8508c3
-  Yoga Studio : 4bf58dd8d48988d102941735

In [14]:
CLIENT_ID = 'U3L0E5VI14H52DFV4DN2Y5MXHUBNTDUPNLU0JI00FLDBWGUM' # your Foursquare ID
CLIENT_SECRET = 'C5R3QORGYXLQKOZ4124QLZNAFH4A41V15ALUDQJVDABB1X0V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
# Gym / Fitness Center : 4bf58dd8d48988d175941735
categoryId = '4bf58dd8d48988d175941735'
LIMIT = 500
radius = 300000

In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, categoryId, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=U3L0E5VI14H52DFV4DN2Y5MXHUBNTDUPNLU0JI00FLDBWGUM&client_secret=C5R3QORGYXLQKOZ4124QLZNAFH4A41V15ALUDQJVDABB1X0V&ll=39.0694616,-94.5560807163324&v=20180605&categoryId=4bf58dd8d48988d175941735&radius=300000&limit=500'

In [16]:
results = requests.get(url).json()
# results

In [17]:
venues = results['response']['venues']

In [18]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.rename(columns={'location.city':'City', 'location.lat':'lat', 'location.lng':'lng', 'location.state':'State'}, inplace=True)

In [19]:
# nearby_venues.groupby(['City','State']).agg({['City','State'] : ['count']})
nearby_venues.groupby(['City','State']).agg({'City' : ['count']})

City
                        count
City              State      
Bonner Springs    KS        1
Excelsior Springs MO        1
Fort Leavenworth  KS        1
Independence      MO        1
Kansas City       KS        1
                  MO       19
Lawrence          KS        3
Lees Summit       MO        3
Lenexa            KS        2
Liberty           MO        1
Merriam           KS        1
Mission           KS        1
North Kansas City MO        1
Olathe            KS        4
Overland Park     KS        6
Parkville         MO        1
Prairie Village   KS        1
Westwood          KS        2

In [21]:
# create map of Overland Park using latitude and longitude values
map_kc = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
# for lat, lng, name in zip(nearby_venues['location.lat'], nearby_venues['location.lng'], nearby_venues['name']):
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kc)  
    
kc_center=[latitude, longitude]
folium.Marker(kc_center).add_to(map_kc)
folium.Circle(kc_center, radius=5000, fill=False, color='white').add_to(map_kc)
folium.Circle(kc_center, radius=10000, fill=False, color='white').add_to(map_kc)
folium.Circle(kc_center, radius=15000, fill=False, color='white').add_to(map_kc)
folium.Circle(kc_center, radius=20000, fill=False, color='white').add_to(map_kc)
folium.Circle(kc_center, radius=25000, fill=False, color='white').add_to(map_kc)
folium.Circle(kc_center, radius=30000, fill=False, color='white').add_to(map_kc)

map_kc